In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard

/home/joseph/anaconda3/envs/idermato/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joseph/anaconda3/envs/idermato/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joseph/anaconda3/envs/idermato/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/joseph/anaconda3/envs/idermato/lib

In [9]:
lrs = {
        'epoch': [],
        'lr': []
    }
def scheduler(epoch):
    if epoch < 10:
        lr = 0.001
    elif epoch % 10 == 0:
        lr = 0.001 * np.exp(0.015 * (10 - epoch))
    else:
        lr = lrs['lr'][epoch-1]
    lrs['epoch'].append(epoch +1)
    lrs['lr'].append(lr)
    return lr

In [10]:
DIR = './dataset/'

In [11]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')
val_generator = datagen.flow_from_directory(
    DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 442 images belonging to 5 classes.
Found 109 images belonging to 5 classes.


In [12]:
inputs = layers.Input(shape=(150, 150, 3))

cv1 = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
pool1 = layers.MaxPooling2D((2, 2))(cv1)

cv2 = layers.Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = layers.MaxPooling2D((2, 2))(cv2)

cv3 = layers.Conv2D(64, (3, 3), activation='relu')(pool2)
pool3 = layers.MaxPooling2D((2, 2))(cv3)

cv4 = layers.Conv2D(128, (3, 3), activation='relu')(pool3)
pool4 = layers.MaxPooling2D((2, 2))(cv4)

flatten = layers.Flatten()(pool4)

dropout = layers.Dropout(0.5)(flatten)

dense1 = layers.Dense(512, activation='relu')(dropout)

outputs = layers.Dense(5, activation='softmax')(dense1)

model = Model(inputs=inputs, outputs=outputs)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 64)        0   

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.optimizer.lr = 1e-3

In [14]:
lr = LearningRateScheduler(scheduler, verbose=1)

tensordboard = TensorBoard(
    log_dir='my_log_dir',
    histogram_freq=1,
#     embeddings_freq=1,
)

callbacks = [tensordboard, lr]

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/20
 1/14 [=>............................] - ETA: 5:05 - loss: 1.6241 - acc: 0.3125

/home/joseph/anaconda3/envs/idermato/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/14 [====================>.........] - ETA: 33s - loss: 2.1493 - acc: 0.2293